## Chat myPDF : RAG with LangChain, Ollama, and FAISS Vector Store

### Document Ingestion


In [78]:
# pip install -U langchain langchain-community faiss-cpu langchain-huggingface pymupdf tiktoken langchain-ollama python-dotenv

In [35]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

load_dotenv()

True

### Document Loader

In [36]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("./PDF_dataset/gym supplements/1. Analysis of Actual Fitness Supplement.pdf")

docs = loader.load()

In [37]:
doc = docs[0]
#print(doc.page_content)

In [38]:
import os

pdfs = []
for root, dirs, files in os.walk('PDF_dataset'):
    # print(root, dirs, files)
    for file in files:
        if file.endswith('.pdf'):
            pdfs.append(os.path.join(root, file))

In [39]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    pages = loader.load()

    docs.extend(pages)

In [40]:
len(docs)

64

### Document Chunking

In [41]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = text_splitter.split_documents(docs)

In [42]:
len(docs), len(chunks)

(64, 311)

In [43]:
len(docs[0].page_content), len(chunks[0].page_content)

(4339, 981)

In [44]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

len(encoding.encode(docs[0].page_content)), len(encoding.encode(chunks[0].page_content))

(968, 294)

### Document Vector Embedding

In [79]:
# First, pull the embedding model from Ollama
# Run this command in your terminal: ollama pull nomic-embed-text
# Or uncomment and run the following cell to pull it programmatically:

import subprocess
result = subprocess.run(['ollama', 'pull', 'nomic-embed-text'], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)


Exception in thread Thread-19 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\Al Rehman Computers\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\Al Rehman Computers\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Al Rehman Computers\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1597, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\Al Rehman Computers\AppData\Local\Programs\Python\Python311\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 541: character maps to <undefined>


In [46]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [47]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")

single_vector = embeddings.embed_query("this is some text data")


In [48]:
len(single_vector)

768

In [49]:
index = faiss.IndexFlatL2(len(single_vector))
index.ntotal, index.d

(0, 768)

In [50]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [51]:
len(chunks)

311

In [52]:
# help(vector_store)

In [53]:
ids = vector_store.add_documents(documents=chunks)

In [54]:
vector_store.index_to_docstore_id
len(ids)

311

In [55]:
# # store vector database
# db_name = "health_supplements"
# vector_store.save_local(db_name)

# # load vector database
# new_vector_store = FAISS.load_local(db_name, embeddings=embeddings, allow_dangerous_deserialization=True)
# len(new_vector_store.index_to_docstore_id)

### Retreival

In [56]:
question = "what is used to gain muscle mass?"
docs = vector_store.search(query=question, search_type='similarity')

for doc in docs:
    print(doc.page_content)
    print("\n\n")

acids than traditional protein sources. Its numerous benefits have made it a popular choice
for snacks and drinks among consumers [3]. Another widely embraced supplement is
caffeine, which is found in many sports and food supplements. Caffeine reduces perceived
effort, minimizes fatigue and pain, and proves to be effective for endurance and high-
intensity activities, which is the choice of consumers [4].
Creatine monohydrate is another well-known supplement used to gain muscle mass
and support performance and recovery. It is known not to increase fat mass and remains
effective even when taken in recommended doses [5]. Despite its popularity in the fitness
Foods 2024, 13, 1424. https://doi.org/10.3390/foods13091424
https://www.mdpi.com/journal/foods



and strength gain among men. We detected more prevalent protein and creatine supplementation
among younger compared to older ﬁtness center users, whereas the opposite was found for vitamin
supplementation. Other authors made similar obse

In [57]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs = {'k': 3, 
                                                                          'fetch_k': 100,
                                                                          'lambda_mult': 1})

In [58]:
docs = retriever.invoke(question)

for doc in docs:
    print(doc.page_content)
    print("\n\n")


acids than traditional protein sources. Its numerous benefits have made it a popular choice
for snacks and drinks among consumers [3]. Another widely embraced supplement is
caffeine, which is found in many sports and food supplements. Caffeine reduces perceived
effort, minimizes fatigue and pain, and proves to be effective for endurance and high-
intensity activities, which is the choice of consumers [4].
Creatine monohydrate is another well-known supplement used to gain muscle mass
and support performance and recovery. It is known not to increase fat mass and remains
effective even when taken in recommended doses [5]. Despite its popularity in the fitness
Foods 2024, 13, 1424. https://doi.org/10.3390/foods13091424
https://www.mdpi.com/journal/foods



and strength gain among men. We detected more prevalent protein and creatine supplementation
among younger compared to older ﬁtness center users, whereas the opposite was found for vitamin
supplementation. Other authors made similar obse

In [80]:


question = "what is used to reduce weight?"
# question = "what are side effects of supplements?"
# question = "what are the benefits of supplements?"
# question = "what are the benefits of BCAA supplements?"
docs = retriever.invoke(question)
# for doc in docs:
#     print(doc.page_content)
#     print("\n\n")



### RAG with LLAMA 3.2 on OLLAMA

In [81]:
# %pip install langchainhub


In [65]:
from langchainhub import Client
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

In [67]:
model = ChatOllama(model="llama3.2", base_url="http://localhost:11434")

model.invoke("hi")

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-12-25T09:18:15.3732042Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8724000300, 'load_duration': 6417344200, 'prompt_eval_count': 26, 'prompt_eval_duration': 1339218500, 'eval_count': 8, 'eval_duration': 936238900, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--019b54cd-1355-72f3-adbd-7a5ab2d1fa25-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [ ]:
# Optional: Pull prompt from LangChain Hub (not used since we define custom prompt below)
# hub_client = Client()
# prompt = hub_client.pull("rlm/rag-prompt")

In [68]:
prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [73]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# print(format_docs(docs))

In [ ]:
rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model 
    | StrOutputParser()

    jbnn
)

In [82]:
# question = "what is used to gain muscle mass?"
# question = "what is used to reduce weight?"
# question = "what are side effects of supplements?"
# question = "what are the benefits of supplements?"
question = "what are the benefits of BCAA supplements?"

# question = "what is used to increase mass ?"

output = rag_chain.invoke(question)
print(output)


I don't know. The retrieved context does not provide information on the benefits of BCAA supplements. It discusses various aspects of dietary supplements in general, but there is no specific mention of BCAA supplements or their benefits.
